## Modelo de Regressão - XGBoost

### Serão testados 3 versões do algoritmo XGBoost:
#### v1 - uso de 10 estimadores
#### v2 - uso da estruta DMatrix para otimização de memória e velocidade de treinamento
#### v3 - uso do GridSelection para seleção dos melhores Hyperparâmetros

## Métricas baseline

#### RMSE = 4.96
#### r2 = 0.76

## Nota

#### Por se tratar por um modelo de baixa complexidade de alta correlação da variável target com a feature 'distance_km', é bem provável que o modelo baseline se saia melhor do que algumas versões.

In [ ]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

In [1]:
# Imports
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error as MSE
import xgboost as xgb
import tools
from datetime import datetime as dt
import pickle
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Fabiano Fonseca - MLE" --iversions

### Carregando dataset

In [ ]:
size_dataset = 1000000
size_float = 32

In [ ]:
csvFileName = "../data/data_fe_" + str(size_dataset) + "_20220320.csv"
df = pd.read_csv(csvFileName)

In [ ]:
df.info()

In [ ]:
if (size_float == 32):
    df = tools.reduce_size_features(df, verbose=True)
    df.info()

In [ ]:
# Shape
df.shape

In [ ]:
df.head()

### Seleciona as Features para serem testadas na criação do Modelo

In [ ]:
# selecionando todas features para experimentação
X = df[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count','hour','day_of_week','day_of_month','week','month','hour_norm','cos_hour','sin_hour','day_of_week_norm','cos_day_of_week','sin_day_of_week','day_of_month_norm','cos_day_of_month','sin_day_of_month','week_norm','cos_week','sin_week','distance_km']]

# Alvo
Y = df[['fare_amount']].values

#split para treino
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

### v1 - uso de 10 estimadores

In [ ]:
xgb_r = xgb.XGBRegressor(objective ='reg:squarederror',
                  n_estimators = 10, seed = 123)
  
xgb_r.fit(X_train, y_train)
pred = xgb_r.predict(X_test)  

In [ ]:
# RMSE Computation
rmse = np.sqrt(MSE(y_test, pred))
print("RMSE : % f" %(rmse))

In [ ]:
print('R2Score: ',r2_score(y_test, pred))

### v2 - uso da estruta DMatrix para otimização de memória e velocidade de treinamento

In [ ]:
train_dmatrix = xgb.DMatrix(data = X_train, label = y_train)
test_dmatrix = xgb.DMatrix(data = X_test, label = y_test)

param = {"booster":"gblinear", "objective":"reg:squarederror"}
  
xgb_r = xgb.train(params = param, dtrain = train_dmatrix, num_boost_round = 10)
pred = xgb_r.predict(test_dmatrix)

In [ ]:
# Calcula RMSE
rmse = np.sqrt(MSE(y_test, pred))
print("RMSE : % f" %(rmse))

In [ ]:
# Calcula r2
print('R2Score: ',r2_score(y_test, pred))

### v3 - uso do GridSelection para seleção dos melhores Hyperparâmetros

In [ ]:
param = {'n_estimators':[100], 'max_depth':range(3,11), 'eta':[0.05,0.1]}
xgr = xgb.XGBRegressor(objective ='reg:squarederror')

# RMSE
gs = GridSearchCV(estimator=xgr,param_grid=param,scoring='neg_root_mean_squared_error',cv=5,verbose=3)
gs.fit(X_train,y_train)

In [ ]:
# Mostrando o melhor hyperparâmetro
# dependendo da máquina, vai demorar algumas horas.
print(gs.best_params_)
print(gs.best_score_)

In [ ]:
#{'eta': 0.05, 'max_depth': 10, 'n_estimators': 100}
#-3.4029746532440184

#coloque aaui o resultado o melhor hyperparâmetros
xgr = xgb.XGBRegressor(objective ='reg:squarederror', eta=0.05,max_depth=10,n_estimators=100)
xgr.fit(X_train,y_train)
pred = xgr.predict(X_test)

In [ ]:
print('RMSE: ',MSE(y_test,pred,squared=False))

In [ ]:
print('R2Score: ',xgr.score(X_test,y_test))

#### Seleção das features que mais explicam a variabilidade da variável alvo.

In [ ]:
sorted_idx = xgr.feature_importances_.argsort()
plt.figure(figsize=(10, 10))
plt.xlabel("Xgboost Feature Importance")
display(plt.barh(X_test.columns[sorted_idx], xgr.feature_importances_[sorted_idx]))

## Escolha do modelo

#### A versão 3 com uso do GridSelection para melhor escolha do hyperparâmetro usado apresentou o melhor resultado sobre o
#### modelo baseline.

### Baseline: RMSE = 4.96  / r2 = 0.76
### XGBoost v3: RMSE = 3.36  / r2 = 0.88

#### Serializando o modelo

In [ ]:
pickle.dump(xgr, open('../modelo/modelo_'+ dt.now().strftime("%Y%m%d") + '.sav','wb'))

### Fim. 

In [2]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [4]:
conda list > '..\data\requirements_raw.txt'


Note: you may need to restart the kernel to use updated packages.
